In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import logging
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import glob
import os

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
affvids_dir = '/home/eli/Documents/AffVids_20180104/'

task_log_csvs = 'editedlogfiles'

TASK_ONSET_DELAY = 3

AFFVID_FILENAME_TEMPLATE = 'sub-%02d_task-AffVids_run-%02d_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'

AFFVIDS_FILE = 'data/affvids2018.db'

def affvid_filename(subject, run):
    return affvids_dir + 'nifti/' + (AFFVID_FILENAME_TEMPLATE % (subject, run))

class TaskElement:
    def __init__(self, task, start, end):
        def round_off_time(t):
            if t is not None:
                return round(t + TASK_ONSET_DELAY)
            else:
                return None
        self.task = task
        self.start_time = round_off_time(start)
        self.end_time = round_off_time(end)

def parse_task_lines(lines):
    for (i, line) in enumerate(lines):
        cols = line.split(' ')
        task = cols[0]
        start_time = float(cols[9])
        end_time = float(cols[10])
        logging.info('Task %d %s started at %f, ended at %f', i, task, start_time, end_time)
        yield TaskElement(task, start_time, end_time)

def rest_tasks(tasks):
    logging.info('Rest 0 ended at %f', tasks[0].start_time - 1)
    yield TaskElement('rest', 0, tasks[0].start_time - 1)
    for i in range(1, len(tasks)):
        rest_start = tasks[i-1].end_time + 1
        rest_end = tasks[i].start_time - 1        
        logging.info('Rest %d started at %f, ended at %f', i, rest_start, rest_end)
        yield TaskElement('rest', rest_start, rest_end)
    logging.info('Rest %d started at %f', len(tasks),  tasks[-1].end_time + 1)
    yield TaskElement('rest', tasks[-1].end_time + 1, None)
        
def read_tasks(task_csv):
    with open(task_csv, 'r') as task_csv_file:
        header = task_csv_file.readline().split(' ')
        subject = int(header[4])
        run = int(header[6])
        logging.info('Subject %d, run %d', subject, run)
        task_lines = list(parse_task_lines(task_csv_file.readlines()))
        task_lines += list(rest_tasks(task_lines))
        task_lines = sorted(task_lines, key=lambda t: t.start_time)
        for (i, task) in enumerate(task_lines):
            result = niidb.FMriActivationBlock(zscore=False)
            result.subject = subject
            result.run = run
            result.task = task.task
            result.block = i
            result.start_time = task.start_time
            result.end_time = task.end_time
            yield result

In [5]:
if not os.path.exists(AFFVIDS_FILE):
    affvids_db = niidb.FMriActivationsDb(AFFVIDS_FILE)

    for task_csv in glob.glob(affvids_dir + task_log_csvs + '/*.txt'):
        for block in read_tasks(task_csv):
            block.filename = affvid_filename(block.subject, block.run)
            affvids_db.upsert(block)

04/26/2018 10:48:50 Subject 23, run 1
04/26/2018 10:48:50 Task 0 spider_low_3.m4v started at 22.143000, ended at 42.958000
04/26/2018 10:48:50 Task 1 spider_low_1.m4v started at 65.932000, ended at 86.213000
04/26/2018 10:48:50 Task 2 heights_low_5.m4v started at 109.168000, ended at 129.200000
04/26/2018 10:48:50 Task 3 heights_low_1.m4v started at 154.155000, ended at 175.188000
04/26/2018 10:48:50 Task 4 social_low_5.m4v started at 198.127000, ended at 218.154000
04/26/2018 10:48:50 Task 5 social_high_2.mov started at 243.080000, ended at 264.049000
04/26/2018 10:48:50 Task 6 spider_high_5.m4v started at 288.002000, ended at 308.663000
04/26/2018 10:48:50 Task 7 spider_high_3.mov started at 331.623000, ended at 351.740000
04/26/2018 10:48:50 Task 8 heights_high_5.mov started at 375.660000, ended at 395.661000
04/26/2018 10:48:50 Task 9 heights_high_2.mov started at 418.598000, ended at 439.466000
04/26/2018 10:48:50 Task 10 social_high_6.mov started at 462.402000, ended at 482.93600

In [6]:
logging.info('Finished building NiiDb out of AffVids dataset in %s', affvids_dir)

04/26/2018 10:50:30 Finished building NiiDb out of AffVids dataset in /home/eli/Documents/AffVids_20180104/
